In [1]:
import json
from PIL import Image
from IPython.display import display
import numpy as np
import pandas as pd
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
# from torch.nn.utils.rnn import pad_sequence
# import torch

import nltk, csv, collections
nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_recall_fscore_support
from scipy.sparse import hstack
import spacy

from transformers import pipeline, AutoModelWithLMHead, AutoTokenizer

nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hisha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# importing the data
datafolder = '../../data/hateful_memes/'
train = datafolder+'train.jsonl'
test = datafolder+'test_seen.jsonl'
dev = datafolder+'dev_seen.jsonl'
# Load the data from the JSON file
df_train = pd.read_json(train, lines = True)
df_dev = pd.read_json(dev, lines = True)
df_test = pd.read_json(test, lines = True)

In [3]:
def preprocess_row(row):
    text = row['text']
    doc = nlp(text)
    tokens = []
    for token in doc:
        pos = token.pos_
        lemma = token.lemma_
        tokens.append((token.text, lemma, pos))
    row['tokens'] = " ".join([t[0] for t in tokens])
    row['lemmas'] = " ".join([t[1] for t in tokens])
    row['upos'] = " ".join([t[2] for t in tokens])
    return row

In [4]:
df_train = df_train.apply(preprocess_row, axis=1)
df_dev = df_dev.apply(preprocess_row, axis=1)
df_test = df_test.apply(preprocess_row, axis=1)


# basline

In [6]:
# importing the data
datafolder = '../../data/hateful_memes/'
train = datafolder+'train_with_features.csv'
test = datafolder+'test_with_features.csv'
dev = datafolder+'dev_with_features.csv'
df_train = pd.read_csv(train, skip_blank_lines=False)
df_dev = pd.read_csv(dev, skip_blank_lines=False)
df_test = pd.read_csv(test, skip_blank_lines=False)

In [7]:
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(), analyzer='word', ngram_range=(1, 1)) # to build 1n-grams from the word ==> BoW
                
X_train = vectorizer.fit_transform(df_train.tokens)
X_dev = vectorizer.transform(df_dev.tokens) 
X_test = vectorizer.transform(df_test.tokens)

Y_train = df_train.label.values
Y_dev = df_dev.label.values
Y_test = df_test.label.values

In [8]:
clf_svc = LinearSVC(max_iter=100000, C = 10,random_state =456)  # TODO: Hyperparms tuning
clf_svc.fit(X_train, Y_train)
Y_pred = clf_svc.predict(X_dev)

In [9]:
results = pd.DataFrame(
    [list(precision_recall_fscore_support(Y_dev, Y_pred, average='macro')[:3])],
    columns=['precision', 'recall', 'F1'])
print("Text classifier results using only n1 BoW")
results

Text classifier results using only n1 BoW


,precision,recall,F1
0,0.541398,0.535797,0.520697


In [10]:
Y_pred2 = clf_svc.predict(X_test)

In [12]:

df_dev['baseline_svm_rbf_kernelC10'] = Y_pred
df_test['baseline_svm_rbf_kernelC10'] = Y_pred2

df_dev.to_csv(datafolder+'dev_with_features.csv', index=False)
df_test.to_csv(datafolder+'test_with_features.csv', index=False)